In [141]:
import pandas as pd

In [142]:
sbt = pd.read_csv("seattle_bus_times.csv")

In [168]:
sbt.iloc[0:5,1:3]

,VEHICLE_ID,RTE
0,6201,C
1,6201,C
2,6201,C
3,6201,C
4,6201,C
